<a href="https://colab.research.google.com/github/bnik6789/EEGData/blob/main/Copy_of_read_portugal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyEDFlib mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 25.8 MB/s eta 0:00:00


In [88]:
%%sh
i=1
mkdir "/content/rar_files"
while [ "$i" -le 50 ]; do
    wget -q "http://dataset.isr.uc.pt/ISRUC_Sleep/subgroupI/$i.rar"  -P "/content/rar_files"
    i=$(( i + 1 ))
done

mkdir: cannot create directory ‘/content/rar_files’: File exists


In [97]:
%%sh
i=3
mkdir '/content/data'
while [ "$i" -le 50 ]; do
    mkdir "/content/data/s$i"
    unrar e "/content/rar_files/$i.rar" "/content/data/s$i"
    i=$(( i + 1 ))
done


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/rar_files/3.rar

Extracting  /content/data/s3/3.rec                                         0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 98% 99%  OK

mkdir: cannot create directory ‘/content/data’: File exists


In [4]:
from pyedflib import highlevel
import pyedflib as plib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne

In [5]:
import pyedflib
from datetime import datetime, date

#new_rec = pyedflib.EdfWriter('test.rec', 3)
f = plib.EdfReader("/content/data/s1/1.rec")

headers = np.array(f.getSignalHeaders())
names = [v for channel in headers for k, v in channel.items() if k == 'transducer'  ]

get_eeg_index = lambda x: [i for i in range(len(x)) if 'EEG' in x[i].upper() and 'EEG2' not in x[i].upper() ]
eeg_index = get_eeg_index(names)

signal = np.array([f.readSignal(i) for i in eeg_index])


  #new_rec.setSignalHeader(f.readSignal(i), headers[i])

#f.close()
#new_rec.close()

In [9]:
with open('sub1.npy', 'wb') as fil:

    np.save(fil, signal)

In [15]:
%%sh
i=1
mkdir '/content/final_data'
while [ "$i" -le 2 ]; do
    mkdir "/content/final_data/s$i" #wget "http://dataset.isr.uc.pt/ISRUC_Sleep/subgroupI/$i.rar"
    #unrar e "/content/$i.rar" "/content/data/s$i"
    i=$(( i + 1 ))
done

mkdir: cannot create directory ‘/content/final_data’: File exists
mkdir: cannot create directory ‘/content/final_data/s1’: File exists


In [105]:
%%sh
i=1
mkdir "/content/eeg_data/"
while [ "$i" -le 50 ]; do
    mkdir  "/content/eeg_data/s$i"
    i=$(( i + 1 ))
done

mkdir: cannot create directory ‘/content/eeg_data/’: File exists


In [83]:
def make_sleep_data1(f, annot, split):
  #signals = np.array([f.readSignal(sig) for sig in range(12)])
  #signals = signals * 1e-6
  headers = np.array(f.getSignalHeaders())
  names = [v for channel in headers for k, v in channel.items() if k == 'transducer'  ]

  get_eeg_index = lambda x: [i for i in range(len(x)) if 'EEG' in x[i].upper() and 'EEG2' not in x[i].upper() ]
  eeg_index = get_eeg_index(names)
  signals = np.array([f.readSignal(i) for i in eeg_index])

  # get rid of the meta data at end of file
  annot = annot.dropna(subset=['Epoch']).copy()

  # get label vector
  annot['Events'] = annot['Events'].astype('str')

  apnea = ["CA", "OA"]
  annot['Labels'] = annot['Events'].apply(lambda x: 1 if any(code in x for code in apnea) else 0 )

  labels = np.array(annot["Labels"])

  if not split:
    return signals, labels

  else:
    samples = 200 * 30
    signals = signals.reshape((12, int(5280000/samples), samples))
    return signals, labels


In [108]:
num_patients = 50

for i in range(30,num_patients+1):
  file_path = f'/content/data/s{i}/'

  final_path = f'/content/eeg_data/s{i}/'
  annot_temp = pd.read_excel(file_path + f'{i}_1.xlsx', usecols=range(9), na_values='')
  eeg = plib.EdfReader(file_path + f'{i}.rec')

  signals, labels = make_sleep_data1(eeg, annot_temp, split = False)

  with open(final_path + f'sub{i}.npy', 'wb') as fil: np.save(fil, signal)
  np.savetxt(final_path + f"sub{i}_labels.csv", labels, delimiter=",")

  eeg.close()


In [110]:
import shutil
shutil.make_archive('eeg_data','zip','/content/eeg_data')

'/content/eeg_data.zip'

In [ ]:
for i in range(len(annot)):
  #print(annot['Events'][i])
  if annot['Events'][i] == annot['Events'][i]:
    annot['Events'][i] = annot['Events'][i].split(",")


<ipython-input-39-480425a035f3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot['Events'][i] = annot['Events'][i].split(",")


In [ ]:
apnea = ["CA", "OA"]
for i in range(len(annot)):
  #print(annot['Events'][i])
  if annot['Events'][i] == annot['Events'][i]:

    if any(code in annot['Events'][i] for code in apnea):
      annot['Events'][i] = 1
    else:
      annot['Events'][i] = 0
  else:
    annot['Events'][i] = 0

<ipython-input-41-ca74a4bc1393>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot['Events'][i] = 0
<ipython-input-41-ca74a4bc1393>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot['Events'][i] = 0
<ipython-input-41-ca74a4bc1393>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot['Events'][i] = 1


In [ ]:
labels = np.array(annot["Events"])

In [ ]:
import torch

In [ ]:
labels = np.array(list(labels))

In [ ]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
torch.tensor(labels)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
signals

array([[ 3.81475547e-10, -6.52323186e-08, -5.09269856e-07, ...,
        -1.19897765e-06, -1.16540780e-06, -1.07385367e-06],
       [ 3.81475547e-10, -1.59838254e-07, -1.15548943e-06, ...,
         2.31975280e-06,  2.34264134e-06,  2.04890517e-06],
       [ 3.81475547e-10, -6.98100252e-08, -5.31395438e-07, ...,
        -1.16922255e-06, -1.01892119e-06, -8.08346685e-07],
       ...,
       [ 1.32753491e-09, -1.98731975e-06, -6.63369192e-06, ...,
         5.13092241e-06,  5.48139162e-06,  5.20526436e-06],
       [ 5.95101854e-10,  3.95921263e-06,  3.90000000e-05, ...,
        -2.08285649e-08, -3.63012131e-08,  5.95101854e-10],
       [ 9.56237125e-07,  3.30000000e-05,  2.29466697e-06, ...,
        -8.91279469e-08, -1.05241474e-07, -4.48157473e-08]])

In [ ]:
samples = 200 * 30

In [ ]:
5280000/samples

880.0

In [ ]:
signals.reshape((12, int(5280000/samples), samples)).shape

(12, 880, 6000)

In [ ]:
signals.shape

(12, 5280000)

In [ ]:
test = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]).reshape((3,8))

In [ ]:
test = test.reshape(3,2,4)

In [ ]:
test

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8]],

       [[ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[17, 18, 19, 20],
        [21, 22, 23, 24]]])

In [ ]:
test.shape

(3, 2, 4)

In [ ]:
annot["Epoch"]

0        1.0
1        2.0
2        3.0
3        4.0
4        5.0
       ...  
875    876.0
876    877.0
877    878.0
878    879.0
879    880.0
Name: Epoch, Length: 880, dtype: float64

In [ ]:
# channel, total time slices, measurements

In [ ]:
def make_sleep_data(f, annot, split):
  signals = np.array([f.readSignal(sig) for sig in range(12)])
  signals = signals * 1e-6

  # get rid of the meta data at end of file
  for i in range(len(annot)):
    if annot["Epoch"][i] != annot["Epoch"][i]
      annot = annot.drop([i])
  # get label vector
  for i in range(len(annot)):
    if annot['Events'][i] == annot['Events'][i]:
      annot['Events'][i] = annot['Events'][i].split(",")

  apnea = ["CA", "OA"]
  for i in range(len(annot)):
    if annot['Events'][i] == annot['Events'][i]:

      if any(code in annot['Events'][i] for code in apnea):
        annot['Events'][i] = 1
      else:
        annot['Events'][i] = 0
    else:
      annot['Events'][i] = 0

    labels = np.array(annot["Events"])
    labels = np.array(list(labels))

    if !split:
      return signals, labels

    else:
      samples = 200 * 30
      signals = signals.reshape((12, int(5280000/samples), samples))
      return signals, labels


